In [1]:
import os
os.chdir('../../scripts/external-data-download/')

In [2]:
OUTPUT_RELATIVE_PATH = "../../data/"
def create_path():
    """
        
    """
    path = ["external-raw-data", "curated/external-data", "raw/external-data"]
    for target_dir in path:
        if not os.path.exists(OUTPUT_RELATIVE_PATH + target_dir):
            os.makedirs(OUTPUT_RELATIVE_PATH + target_dir)
    print('Already Create Paths')

create_path()

Already Create Paths


In [3]:
import gdown
url_suburb_to_postcode = 'https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv'
output_convert = '../../data/external-raw-data/suburb-to-postcode.csv'
gdown.download(url_suburb_to_postcode, output_convert, quiet=False)

Downloading...
From: https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv
To: /Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/data/external-raw-data/suburb-to-postcode.csv
100%|██████████| 7.56M/7.56M [00:05<00:00, 1.39MB/s]


'../../data/external-raw-data/suburb-to-postcode.csv'

In [4]:
import gdown
url_convert = 'https://drive.google.com/uc?id=1phfJSEVdWOdt8PQGtw_QmWkT2uYLk_j1'
output_convert = '../../data/external-raw-data/lga_postcode_table.xlsx'
gdown.download(url_convert, output_convert, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1phfJSEVdWOdt8PQGtw_QmWkT2uYLk_j1
To: /Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/data/external-raw-data/lga_postcode_table.xlsx
100%|██████████| 101k/101k [00:00<00:00, 352kB/s]


'../../data/external-raw-data/lga_postcode_table.xlsx'

In [2]:
from utils import download
download.download_all()

http://house.speakingsame.com/suburbtop.php?sta=vic&cat=Median%20household%20income&name=Weekly%20income&page=0
9
['Rank', 'Suburb', 'Value']


/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/utils.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index=True)


http://house.speakingsame.com/suburbtop.php?sta=vic&cat=Median%20household%20income&name=Weekly%20income&page=1
9
['Rank', 'Suburb', 'Value']


/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/utils.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index=True)


http://house.speakingsame.com/suburbtop.php?sta=vic&cat=Median%20household%20income&name=Weekly%20income&page=2
9
['Rank', 'Suburb', 'Value']


...

/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/utils.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index=True)


http://house.speakingsame.com/suburbtop.php?sta=vic&cat=Median%20household%20income&name=Weekly%20income&page=12
9
['Rank', 'Suburb', 'Value']


/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/utils.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index=True)


http://house.speakingsame.com/suburbtop.php?sta=vic&cat=Median%20household%20income&name=Weekly%20income&page=13
9
['Rank', 'Suburb', 'Value']


/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/utils.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index=True)


In [2]:
from download import download

download.download_all()

22/09/23 19:06:11 WARN Utils: Your hostname, QuzihandeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.96.115 instead (on interface en0)
22/09/23 19:06:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/23 19:06:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/23 19:06:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/23 19:06:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/23 19:06:14 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/09/23 19:06:14 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/09/23 19:06:14 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/09/23 19:06:14 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/09/23 19:06:14 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
22/09/23 19:06:14 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
